In [88]:
import numpy as np
import random
from tic_env import TictactoeEnv, OptimalPlayer

In [89]:
env = TictactoeEnv()
env.__dict__

{'grid': array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 'end': False,
 'winner': None,
 'player2value': {'X': 1, 'O': -1},
 'num_step': 0,
 'current_player': 'X'}

In [90]:
class QPlayer:
    
    def __init__(self, epsilon, alpha = 0.05, gamma = 0.99, player_name = 'O'):
        self.epsilon = epsilon
        self.alpha = alpha
        self.gamma = gamma
        self.player_name = player_name
        self.states = []
        self.actions = []
        self.states_opponent = []
        self.actions_opponent = []
        self.states_value = dict()
        
    def set_player(self, player_name = 'X', j=-1):
        self.player_name = player_name
        if j != -1:
            self.player_name = 'X' if j % 2 == 0 else 'O'
            
    def empty(self, grid):
        '''return all empty positions'''
        avail = []
        for i in range(9):
            pos = (int(i/3), i % 3)
            if grid[pos] == 0:
                avail.append(i)
        return avail
    
    def get_state_key(self, grid):
        "Transforms the grid configuration into a string -- needed for the key of the dictionary" 
        key = str(grid.reshape(3 * 3))
        return key
    
    def select_optimal_action(self, grid):
        avail_actions = self.empty(grid)
        key = self.get_state_key(grid)
        if key in self.states_value : 
            restricted_vector = self.states_value[key][avail_actions]
            move = np.argmax(restricted_vector)
        else :
            move = self.select_random_action(grid)
        return move
    
    def select_random_action(self,grid):
        actions = self.empty(grid)
        move = random.choice(actions)
        return move
    
    def act(self,grid):
        b = np.random.binomial(1,1-self.epsilon)
        if b == 1 : return self.select_optimal_action(grid)
        else : return self.select_random_action(grid)
        
class QLearning:
    def __init__(self, epsilon1, epsilon2, alpha = 0.05, gamma = 0.99):
        self.q_player = QPlayer(epsilon1, alpha, gamma)
        self.opponent = OptimalPlayer(epsilon2)
        self.env = TictactoeEnv()
        
    def game(self):
        env.reset()
        while not env.end : 
            if self.env.current_player == 'O' : 
                move = self.q_player.act()
                self.env.step(move)
                self.q_player.states.append(self.env.grid)
                self.q_player.actions.append(move)

            else : 
                move = self.opponent.act()
                self.env.step(move)
                self.q_player.states_opponent.append(self.env.grid)
                self.q_player.actions_opponent.append(move)
        FINIRE DI SCRIVERE
        
    def give_reward(self):
        
    def update_q(self, player, states, actions):
        r = self.env.reward(player)
        i = len(actions)-1
        key = self.q_player.get_state_key(self.env.grid)
        self.q_player.states_value[key] = np.zeros([9,1])

        for st in reversed(states)[1:]:
            key_prec = self.q_player.get_state_key(st-1)
            key = self.q_player.get_state_key(st)
            if self.q_player.states_value[key] is None:
                self.q_player.states_value[key] = np.zeros([9,1])
                
            self.q_player.states_value[key][actions[i]] += self.q_player.alpha*(r + self.q_player.gamma*max(self.q_player.states_value[key_prec])-self.q_player.states_value[key][actions[i]])  
            r = 0
            i -= 1
            
            
        
        

In [91]:
np.random.seed(2022)
epsilon = np.random.uniform(0,0.5)
anna = QPlayer(epsilon) 
player_opt_1 = OptimalPlayer(epsilon = 0., player = 'X')

In [92]:
grid = env.grid
env.observe()

(array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 False,
 None)

In [93]:
env.step(anna.select_move(grid))

(array([[1., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 False,
 None)

In [94]:
env.step(player_opt_1.act(grid))

(array([[ 1.,  0.,  0.],
        [ 0., -1.,  0.],
        [ 0.,  0.,  0.]]),
 False,
 None)

In [95]:
env.step(anna.select_move(grid))

(array([[ 1.,  0.,  0.],
        [ 0., -1.,  0.],
        [ 0.,  0.,  1.]]),
 False,
 None)

In [96]:
env.step(player_opt_1.act(grid))

(array([[ 1.,  0., -1.],
        [ 0., -1.,  0.],
        [ 0.,  0.,  1.]]),
 False,
 None)

In [97]:
env.step(anna.select_move(grid))

(array([[ 1.,  0., -1.],
        [ 0., -1.,  0.],
        [ 1.,  0.,  1.]]),
 False,
 None)

In [98]:
env.step(player_opt_1.act(grid))

(array([[ 1.,  0., -1.],
        [-1., -1.,  0.],
        [ 1.,  0.,  1.]]),
 False,
 None)

In [99]:
env.step(anna.select_move(grid))

(array([[ 1.,  0., -1.],
        [-1., -1.,  1.],
        [ 1.,  0.,  1.]]),
 False,
 None)

In [100]:
env.step(player_opt_1.act(grid))

(array([[ 1.,  0., -1.],
        [-1., -1.,  1.],
        [ 1., -1.,  1.]]),
 False,
 None)

In [101]:
env.step(anna.select_move(grid))

(array([[ 1.,  1., -1.],
        [-1., -1.,  1.],
        [ 1., -1.,  1.]]),
 True,
 None)

In [102]:
end

NameError: name 'end' is not defined

In [103]:
env.end

True